In [1]:
import pygame
import pygame_gui
import sys
import numpy as np

pygame-ce 2.4.0 (SDL 2.28.5, Python 3.11.6)


In [2]:
initial_board = np.array([
    [ 4, 6, 7, 1, 2, 7, 6, 4 ],
    [ 8, 8, 8, 8, 8, 8, 8, 8 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 18, 18, 18, 18, 18, 18, 18, 18 ],
    [ 14, 16, 17, 11, 12, 17, 16, 14 ],
])

In [3]:
# piece id to sprite id
def pid2sid(id):
    assert id in range(1, 21), "Piece ID must be between 0 and 20."
    if id > 10:
        id -= 10
        i = 6
    else:
        i = 0

    if id == 1:
        j = 0
    elif id == 2 or id == 3:
        j = 1
    elif id == 4 or id == 5:
        j = 2
    elif id == 6:
        j = 3
    elif id == 7:
        j = 4
    elif id == 8 or id == 9 or id == 10:
        j = 5
    else:
        assert False, "Logic error."

    return i + j

In [4]:
# pid legend:
#       BLACK
# pid:  1      2      3      4      5      6       7       8      9      10
#       queen, king1, king2, rook1, rook2, knight, bishop, pawn1, pawn2, pawn3
# sid:  0      1      1      2      2      3       4       5      5      5

# WHITE
# pid:  11     12     13     14     15     16      17      18     19     20 
#       queen, king1, king2, rook1, rook2, knight, bishop, pawn1, pawn2, pawn3
#       6      7             8             9       10      11

In [5]:
initial_board

array([[ 4,  6,  7,  1,  2,  7,  6,  4],
       [ 8,  8,  8,  8,  8,  8,  8,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [18, 18, 18, 18, 18, 18, 18, 18],
       [14, 16, 17, 11, 12, 17, 16, 14]])

In [6]:
def move2d(dx, dy):
    return lambda pos: (pos[0] + dx, pos[1] + dy)

In [10]:
class ChessGame():
    def __init__(self, board=None, current_player=1):
        self.board = initial_board.copy() if board is None else board
        self.current_player = current_player
        self._move_funcs = { 
            4: self.rook_moves,
            5: self.rook_moves,
            6: self.knight_moves,
            7: self.bishop_moves,
            8: self.unmoved_pawn_moves,
            10: self.pawn_moves
        }
        self._double_pawn_pos = None
        #self._white_positions = self._get_white_positions()
        #self._black_positions = self._get_black_positions()

    # @property
    # def _positions(self):
    #     return self._white_positions + self._black_positions
                       

    def copy(self):
        return ChessGame(board=self.board.copy(), current_player=self.current_player)

    @property
    def opponent(self):
        return self.current_player ^ 3

    def get(self, pos):
        x, y = pos
        return self.board[y][x]

    def set(self, pos, piece):
        x, y = pos
        self.board[y][x] = piece

    def empty(self, pos):
        return self.get(pos) == 0

    def move_piece(self, src, dst):
        self.set(dst, self.get(src))
        self.set(src, 0)

    def end_turn(self):
        self.current_player = self.opponent

        if self._double_pawn_pos is not None:
            if self.get(self._double_pawn_pos) in [9, 19]: # if pawn still on board (not captured)
                # since we just swapped opponents, if there is a 9 or 19 here, current_player
                # must be back to what it was when we played the 9 or 19.
                self.set(self._double_pawn_pos, 20 if self.current_player == 1 else 10) 
            self._double_pawn_pos = None


    def play(self, piece_pos, move_pos):
        assert self.player_piece(piece_pos), "Bad logic: trying to play move from non-player piece."

        x1, y1 = piece_pos
        x2, y2 = move_pos

        piece_type = self.piece_type(self.get(piece_pos))
        
        # Move piece
        if self.get(move_pos) == 0:
            
            if piece_type == 8: # unmoved pawn
                x, y1 = piece_pos
                _, y2 = move_pos
                self.set(piece_pos, 0)
                if abs(y1 - y2) == 2: # two spaces
                    self.set(move_pos, 19 if self.current_player == 1 else 9)
                    self.end_turn()
                    self._double_pawn_pos = move_pos
                    return
                else: # one space
                    self.set(move_pos, 20 if self.current_player == 1 else 10)      

            elif piece_type == 10: # normal pawn
                if self.empty(move_pos) and self.piece_type(self.get((x2, y1))) == 9:
                    self.set((x2, y1), 0)
                self.move_piece(piece_pos, move_pos)
            
            else:
                self.move_piece(piece_pos, move_pos)


        # Capture piece
        else:
            if piece_type == 8: # unmoved pawn
                self.set(piece_pos, 0)
                self.set(move_pos, 20 if self.current_player == 1 else 10)      
            else:
                self.move_piece(piece_pos, move_pos)

        self.end_turn()
        return
    
    
    @staticmethod
    def player_pid_range(player):
        if player == 1: # white
            return range(11, 21)
        elif player == 2: # black
            return range(1, 11)
        else:
            assert False, "1 and 2 are the only valid player ids."

    @staticmethod
    def in_bounds(pos):
        try:
            x, y = pos
            if not x in range(0, 8) or not y in range(0, 8):
                return False
        except:
            assert False, "pos is not a position."

        return True

    # assert pos is a pos
    # return pid at pos if pos contains one of current_player's live pieces
    # else return None
    def player_piece(self, pos):
        assert self.in_bounds(pos), "Position must be in bounds."
        
        if self.get(pos) in self.player_pid_range(self.current_player):
            return self.get(pos) # never 0
        else:
            return False

    def piece_player(self, pos):
        if self.get(pos) in self.player_pid_range(1):
            return 1 
        elif self.get(pos) in self.player_pid_range(2):
            return 2
        else:
            return 0
        
    @staticmethod
    def piece_type(pid):
        assert pid in range(1, 21), "pid must be in range(1, 21)."
        if pid in range(11, 21):
            pid -= 10
        return pid

    def moves_in_bounds(self, movelist):
        return [ move for move in movelist if self.in_bounds(move) ]

    # return all positions that do not contain one of current_player's pieces
    def player_open_moves(self, movelist):
        return [ move for move in movelist if not self.player_piece(move) ]

    # return all positions that do not contain any piece
    def open_moves(self, movelist):
        return [ move for move in movelist if self.get(move) == 0 ]

    def generate_line(self, origin, movement):
        pos = movement(origin)
        
        line = []
        while self.in_bounds(pos) and self.piece_player(pos) == 0:
            line = line + [pos]
            pos = movement(pos)

        if self.in_bounds(pos) and self.piece_player(pos) == self.opponent:
            line = line + [pos]

        return line

    def rook_moves(self, pos):
        return [ move
                for movelist in [ self.generate_line(pos, move2d(dx, dy)) for (dx, dy) in [ (1, 0), (-1, 0), (0, 1), (0, -1) ] ]
                for move in movelist
               ]
        
    def bishop_moves(self, pos):
        return [ move
                for movelist in [ self.generate_line(pos, move2d(dx, dy)) for (dx, dy) in [ (1, 1), (-1, 1), (1, -1), (-1, -1) ] ]
                for move in movelist
               ]


    def knight_moves(self, pos):
        x, y = pos
        return self.player_open_moves(self.moves_in_bounds(
            [ (x + d1, y + d2) for (d1, d2) in 
             [p for x, y in [(1, 2), (2, 1)] for p in [(x, y), (-x, y), (x, -y), (-x, -y)]]            
            ]
        ))

    def pawn_attacks(self, pos):
        x, y = pos
        dy = (-1 if self.current_player == 1 else 1)
        return [
            (x + dx, y + dy) for dx in [-1, 1] if (self.in_bounds((x + dx, y + dy)) and self.piece_player((x + dx, y + dy)) == self.opponent)
        ] + self.en_passant_attacks(pos)

    def en_passant_attacks(self, pos):
        x, y = pos
        dy = (-1 if self.current_player == 1 else 1)
        return [
            (x + dx, y + dy) for dx in [-1, 1] if (self.in_bounds((x + dx, y + dy)) and self.piece_player((x + dx, y)) == self.opponent and self.piece_type(self.get((x + dx, y))) == 9)
        ]
    
    def unmoved_pawn_moves(self, pos):
        x, y = pos
        pawn_move = self.pawn_moves(pos)
        if pawn_move:
            return self.player_open_moves(self.moves_in_bounds(
                pawn_move + [ (x, y + (-2 if self.current_player == 1 else 2)) ]
            ))
        else:
            return None

    def pawn_moves(self, pos):
        x, y = pos
        return self.open_moves(self.moves_in_bounds(
            [ (x, y + (-1 if self.current_player == 1 else 1)) ]
        )) + self.pawn_attacks(pos)
    
    # return list of potential moves (for current player) from pos
    def potential_moves(self, pos):
        if not self.player_piece(pos):
            return None
        else:
            try:
                move_fn = self._move_funcs[self.piece_type(self.get(pos))]
            except:
                print(f'unimplemented {self.player_piece(pos)}')
                return None
            
            return move_fn(pos)

In [ ]:
# Actual game/render loop
# -----------------------

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 400, 400
TILE_SIZE = WIDTH // 8

# Colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)

# Game data
game = ChessGame()
cursor = None
potential_moves = None

# Load sprite sheet
spritesheet = pygame.image.load('ChessPiecesArray.png')
sprite_size = spritesheet.get_size()[1]//2 # height / 2
sprites = [
    pygame.transform.scale(
        spritesheet.subsurface(pygame.Rect(j*sprite_size, i*sprite_size, sprite_size, sprite_size)), # Surface of (i, j)-th sprite
        (TILE_SIZE, TILE_SIZE) # size to resize to
    ) for i in range(2) for j in range(6)
]

# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple Chess")

# cursor col,row position to pixel location
def pos2pix(pos):
    (r, c) = pos
    return (TILE_SIZE*r, TILE_SIZE*c)

# Main game loop
running = True
while running:    
    # CONTROL
    # ------------------------------------------------------------    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                continue
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:  # Left mouse button click
            x, y = event.pos
            ccol = x // TILE_SIZE
            crow = y // TILE_SIZE
            last_cursor = cursor
            cursor = (ccol, crow)
            if potential_moves and cursor in potential_moves:
                game.play(last_cursor, cursor)
                potential_moves = None
            else:
                potential_moves = game.potential_moves(cursor)
                print(potential_moves)

    # UPDATE
    # ------------------------------------------------------------

    # RENDER
    # ------------------------------------------------------------
    for row in range(8):
        for col in range(8):
            # tile colour
            color = (0xd1, 0xe3, 0xff) if (row + col) % 2 == 0 else (0x57, 0x97, 0xff)
            # cursor tile colour
            if (col, row) == cursor and game.player_piece(cursor):
                color = (0x22, 0x88, 0x00)
            # potential_move tile colour
            elif potential_moves and (col, row) in potential_moves:            
                R, G = (0xdd, 0xcc)
                color = (R, G, 0x88) if (row + col) % 2 == 0 else (R, G, 0x44)
            # draw tile
            pygame.draw.rect(screen, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))
            # draw sprites
            if game.board[row][col] in range(1, 21):
                screen.blit(sprites[pid2sid(game.board[row][col])], pos2pix((col, row)))
    
    pygame.display.flip()


pygame.quit()
#sys.exit()        


[]
[(4, 5), (4, 4)]
[(5, 2), (5, 3)]
None
[(0, 2), (0, 3)]
[(4, 6), (3, 5), (2, 4), (1, 3), (0, 2)]
[(1, 2), (0, 2), (1, 3)]
[(1, 3), (2, 4), (3, 5), (4, 6), (5, 7), (1, 1), (2, 0)]


In [13]:
# TODO:
#
# finish adding move_fns for all piece types (queen, king, rook, bishop)
#
# make rooks switch to type 5 after first movement



In [ ]:
# TESTS

In [11]:
# in_bounds tests
assert (ChessGame.in_bounds(p) for p in zip(range(8), range(8)))
assert not ChessGame.in_bounds((4, 8))
assert not ChessGame.in_bounds((8, 4))
assert not ChessGame.in_bounds((20, 20))
assert not ChessGame.in_bounds((3, -1))
assert not ChessGame.in_bounds((-1, 3))
assert not ChessGame.in_bounds((-20, -20))